<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-demos/blob/main/05_question_cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The text and semantic cache
- in memory
- redis
- redis semantic
- GPT cache

In [20]:
!pip install -Uqq openai langchain tiktoken colab-env pymupdf faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 45.1 MB/s eta 0:00:00


In [3]:
import os
import colab_env

Mounted at /content/gdrive


In [10]:
!wget https://manuals.info.apple.com/MANUALS/2000/MA2104/en_US/airpods-3gen-lightning-charging-03405241B-rci.pdf -O /content/gdrive/MyDrive/data/airpods.pdf

--2023-07-26 01:41:55--  https://manuals.info.apple.com/MANUALS/2000/MA2104/en_US/airpods-3gen-lightning-charging-03405241B-rci.pdf
Resolving manuals.info.apple.com (manuals.info.apple.com)... 23.45.100.45
Connecting to manuals.info.apple.com (manuals.info.apple.com)|23.45.100.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1432122 (1.4M) [application/pdf]
Saving to: ‘/content/gdrive/MyDrive/data/airpods.pdf’

/content/gdrive/MyD 100%[===================>]   1.37M  --.-KB/s    in 0.08s   

2023-07-26 01:41:56 (17.9 MB/s) - ‘/content/gdrive/MyDrive/data/airpods.pdf’ saved [1432122/1432122]



In [55]:
!ls -alt /content/gdrive/MyDrive/data/airpods.pdf

-rw------- 1 root root 1432122 Sep 21  2022 /content/gdrive/MyDrive/data/airpods.pdf


In [56]:
import openai
import langchain
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('/content/gdrive/MyDrive/data/airpods.pdf')
docs = loader.load()

reference: https://python.langchain.com/docs/modules/model_io/models/llms/llm_caching

In Memory Cache

In [57]:
from langchain.cache import InMemoryCache

langchain.llm_cache = InMemoryCache()

In [58]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recur_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
split_docs = recur_splitter.split_documents(docs)

In [59]:
openai.api_key=os.getenv('OPENAI_API_KEY')

In [60]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

index_root_dir = '/content/gdrive/MyDrive/index/'

db_faiss = FAISS.from_documents(split_docs, embeddings)
db_faiss.save_local(index_root_dir+'faiss_index_airpods')

In [62]:
index_faiss = FAISS.load_local(index_root_dir+'faiss_index_airpods', embeddings)
retriever = index_faiss.as_retriever(k=1)

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain


llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo', verbose=False)

conversation_chain = ConversationalRetrievalChain.from_llm(
                        llm=llm,
                        retriever=retriever,
                        verbose=False,
                    )

In [64]:
chat_history = []

def conversation(question):
    input = {'question': question, 'chat_history': chat_history}
    result = conversation_chain(input)
    chat_history.append((question, result['answer']))
    print("Question:\n %s \n" % question)
    print("Answer:\n %s \n" % result['answer'])


In [70]:
%%time

conversation("what is the FCC compliance?")

Question:
 what is the FCC compliance? 

Answer:
 FCC compliance refers to the adherence of a device to the regulations and standards set by the Federal Communications Commission (FCC) in the United States. It means that the device has been tested and meets the requirements specified by the FCC for electromagnetic interference (EMI) and radio frequency interference (RFI). Compliance ensures that the device operates within acceptable limits and does not interfere with other electronic devices. 

CPU times: user 124 ms, sys: 7.09 ms, total: 131 ms
Wall time: 3.52 s


In [76]:
%%time

conversation("what is the FCC compliance?")

Question:
 what is the FCC compliance? 

Answer:
 FCC compliance refers to the adherence of a device to the regulations and standards set by the Federal Communications Commission (FCC) in the United States. These regulations are in place to ensure that electronic devices, such as radios, televisions, and other electronic devices, do not cause interference with other devices and operate within acceptable limits. 

CPU times: user 17.4 ms, sys: 1 ms, total: 18.4 ms
Wall time: 878 ms


In [77]:
for (x, y) in list(langchain.llm_cache._cache.keys()):
    print(x)
    # print()

[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "SystemMessage"], "kwargs": {"content": "Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n\u00a9 2022 Apple Inc. All rights reserved. Apple, the Apple logo, AirPods, and Lightning are trademarks\n\n\u2022 \n\u2022 \n\u2022 \n\u2022 \nApple Inc. \nOne Apple Park Way, MS 911-AHW \nCupertino, CA 95014 USA\n\napple.com/hk/contact\nISED Canada\nThis device complies with ISED Canada\n\nthe web at apple.com/au/recycling/batteries. \nIndependent recycling professionals can obtain", "additional_kwargs": {}}}, {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "tell me where I an get a free airpods?", "additional_kwargs": {}}}]
[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"

In [ ]:
# !wget http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb
# !sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2.19_amd64.deb

In [ ]:
!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
!tar -xvf  redis-stack-server.tar.gz
!pip install redis

In [80]:
! ls -alt ./redis-stack-server-6.2.6-v7/bin/redis-stack-server
! ./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes

-rwxr-xr-x 1 1001 123 1864 Apr 24 12:47 ./redis-stack-server-6.2.6-v7/bin/redis-stack-server
Starting redis-stack-server, database path ./redis-stack-server-6.2.6-v7/var/db/redis-stack


In [95]:
import redis

from langchain.memory.chat_message_histories import RedisChatMessageHistory


client = redis.Redis(host='localhost', port=6379)

client.ping()


True

In [96]:
from redis import Redis
from langchain.cache import RedisCache

langchain.llm_cache = RedisCache(redis_=client)

llm_redis = ChatOpenAI(temperature=0, model='gpt-3.5-turbo', verbose=False)

conversation_chain_redis = ConversationalRetrievalChain.from_llm(
                        llm=llm_redis,
                        retriever=retriever,
                        verbose=False,
                    )

In [86]:
chat_history = []

def conversation(question):
    input = {'question': question, 'chat_history': chat_history}
    result = conversation_chain_redis(input)
    chat_history.append((question, result['answer']))
    print("Question:\n %s \n" % question)
    print("Answer:\n %s \n" % result['answer'])


In [88]:
%%time

conversation("what is the FCC compliance?")

Question:
 what is the FCC compliance? 

Answer:
 FCC compliance refers to the adherence of a device to the regulations and standards set by the Federal Communications Commission (FCC) in the United States. These regulations are in place to ensure that electronic devices, such as radios, televisions, and other electronic devices, meet certain requirements for electromagnetic compatibility (EMC) and wireless compliance. Compliance with FCC regulations is necessary for the sale and use of electronic devices in the United States. 

CPU times: user 124 ms, sys: 7.97 ms, total: 132 ms
Wall time: 3.63 s


/usr/local/lib/python3.10/dist-packages/langchain/cache.py:237: UserWarning: NOTE: Generation has not been cached. RedisCache does not support caching ChatModel outputs.
  warnings.warn(


*Generation has not been cached. RedisCache does not support caching ChatModel outputs.*

In [97]:
client.keys()

[b'727d1759db6f508f8745e566191729d1', b'b4b5e406a5cb4d80fc9a35facd0f16e5']

In [92]:
from langchain.llms import OpenAI

llm_redis = OpenAI(temperature=0, verbose=False)

conversation_chain_redis = ConversationalRetrievalChain.from_llm(
                        llm=llm_redis,
                        retriever=retriever,
                        verbose=False,
                    )

In [98]:
%%time

conversation("what is the FCC compliance?")

Question:
 what is the FCC compliance? 

Answer:
 The specific regulations and standards set by the FCC for electromagnetic compatibility (EMC) and wireless compliance are not mentioned in the provided context. 

CPU times: user 38 ms, sys: 5.31 ms, total: 43.3 ms
Wall time: 2.89 s


In [99]:
client.keys()

[b'727d1759db6f508f8745e566191729d1', b'b4b5e406a5cb4d80fc9a35facd0f16e5']

Semantic cache
- https://python.langchain.com/docs/integrations/llms/llm_caching

In [114]:
from langchain.cache import RedisSemanticCache

langchain.llm_cache = RedisSemanticCache(
    redis_url='redis://localhost:6379', embedding=OpenAIEmbeddings(), score_threshold=0.1) #default 0.2

llm_redis_semantic = OpenAI(temperature=0, verbose=False)

conversation_chain_redis_semantic = ConversationalRetrievalChain.from_llm(
                        llm=llm_redis_semantic,
                        retriever=retriever,
                        verbose=False,
                    )

In [115]:
chat_history = []

def conversation(question):
    input = {'question': question, 'chat_history': chat_history}
    result = conversation_chain_redis_semantic(input)
    chat_history.append((question, result['answer']))
    print("Question:\n %s \n" % question)
    print("Answer:\n %s \n" % result['answer'])


In [116]:
%%time

conversation("what is the FCC compliance?")

Question:
 what is the FCC compliance? 

Answer:
  The FCC compliance is that this device complies with part 15 of the FCC rules for radios, televisions, and other electronic devices, and has been tested for EMC compliance. 

CPU times: user 109 ms, sys: 4.77 ms, total: 114 ms
Wall time: 744 ms


In [117]:
langchain.llm_cache._cache_dict

{'cache:0e4d63bf605cd5f4329128fcbe38762d': <langchain.vectorstores.redis.Redis at 0x7aff7bc7e920>}

In [124]:
langchain.llm_cache._cache_dict['cache:0e4d63bf605cd5f4329128fcbe38762d'].similarity_search_with_score(query='what is the FCC compliance')

[(Document(page_content="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nRegulatory Compliance Information\nFCC Compliance\nThis device complies with part 15 of the FCC\n\nradios, televisions, and other electronic devices. \nThis product was tested for EMC compliance\n\ndevice, pursuant to part 15 of the FCC Rules. \nThese limits are designed to provide reasonable\n\nelectromagnetic compatibility (EMC) and \nwireless compliance and negate your authority to\n\nQuestion: what is the FCC compliance?\nHelpful Answer:", metadata={'llm_string': "[('_type', 'openai'), ('frequency_penalty', 0), ('logit_bias', {}), ('max_tokens', 256), ('model_name', 'text-davinci-003'), ('n', 1), ('presence_penalty', 0), ('request_timeout', None), ('stop', None), ('temperature', 0.0), ('top_p', 1)]", 'prompt': "Use the following pieces of context to answer the question at the end. If you don't 

In [109]:
client.keys()

[b'727d1759db6f508f8745e566191729d1',
 b'b4b5e406a5cb4d80fc9a35facd0f16e5',
 b'doc:cache:0e4d63bf605cd5f4329128fcbe38762d:bcf7737bc8764e779426a28d9b4a1644']

GPT cache supports Exact match and similar match

In [125]:
!pip install gptcache -Uqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 2.7 MB/s eta 0:00:00


In [130]:
from gptcache import Cache
from gptcache.manager.factory import manager_factory
from gptcache.processor.pre import get_prompt
from langchain.cache import GPTCache


import hashlib

def init_gptcache(cache_obj: Cache, llm: str):
    hash_llm = hashlib.sha256(llm.encode()).hexdigest()
    cache_obj.init(
        pre_embedding_func = get_prompt,
        data_manager = manager_factory(manager="map", data_dir=f"map_cache_{hash_llm}")
    )

langchain.llm_cache = GPTCache(init_gptcache)

In [131]:
llm = OpenAI(temperature=0, verbose=False)

In [134]:
%%time

llm("what time is it?")

CPU times: user 13.7 ms, sys: 967 µs, total: 14.6 ms
Wall time: 980 ms


'\n\nIt is currently 11:45 AM.'

In [138]:
from gptcache import Cache
from gptcache.adapter.api import init_similar_cache
from langchain.cache import GPTCache

def init_gptcache(cache_obj: Cache, llm: str):
    hash_llm = hashlib.sha256(llm.encode()).hexdigest()
    #faiss and sqlite, SearchDistanceEvaluation
    init_similar_cache(cache_obj=cache_obj, data_dir=f"map_cache_{hash_llm}")

langchain.llm_cache = GPTCache(init_gptcache)

In [142]:
%%time

llm("what time is it?")

CPU times: user 863 ms, sys: 0 ns, total: 863 ms
Wall time: 876 ms


'\n\nIt is currently 11:45 AM.'